In [2]:
import zipfile

In [7]:
# Extract a zip file
with zipfile.ZipFile('database\human+activity+recognition+using+smartphones.zip', 'r') as zip_ref:
    zip_ref.extractall('database\dataset_UCI_HAR')

<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jonas\AppData\Local\Temp\ipykernel_17992\542450039.py:2: SyntaxWarning: invalid escape sequence '\h'
  with zipfile.ZipFile('database\human+activity+recognition+using+smartphones.zip', 'r') as zip_ref:
C:\Users\jonas\AppData\Local\Temp\ipykernel_17992\542450039.py:3: SyntaxWarning: invalid escape sequence '\d'
  zip_ref.extractall('database\dataset_UCI_HAR')


In [8]:
# Extract a zip file
with zipfile.ZipFile(r'database\dataset_UCI_HAR\UCI HAR Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall(r'database\dataset_UCI_HAR')

In [ ]:
dataset_path = r'database\dataset_UCI_HAR\UCI HAR Dataset'